# **Araç Satış Tahminleme**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer





In [ ]:
# Dosyayı yükle
data = pd.read_csv('/content/ikinciYeniReg.csv')
data.head()

,Açılış fiyatı,Kapanış fiyatı,Kaput,SoLÖnÇamurluk,SoLÖnKapı,SoLArkaKapı,SoLArkaÇamurluk,bagaj,SaĞÖnÇamurluk,SaĞÖnKapı,...,Döşeme: Kumaş/Deri Kombinasyonu,Xenon Far,Elektrik Ayarlı Koltuk,Diz Hava Yastığı,Döşeme: Deri,Hafızalı Koltuk,CD Çalarlı Radyo,Döşeme: Alcantara/Deri,4 X 4,CD Değiştiricili Radyo
0,45.0,62.9,B,B,B,D,B,D,B,D,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
1,79.0,95.8,B,D,B,D,B,D,D,D,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
2,49.0,74.5,B,D,B,D,B,D,D,B,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
3,45.0,71.5,B,D,B,B,B,D,B,D,...,VAR,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK
4,29.9,48.5,D,D,B,D,B,D,D,D,...,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK,YOK


In [ ]:
# Kapanış fiyatı sütununu temizleme
data['Kapanış fiyatı'] = data['Kapanış fiyatı'].str.replace('TEKLİF YOK', '0').astype(float)

# Kapanış fiyatı sütunundaki değeri 0 olan satırları silme
data = data[data['Kapanış fiyatı'] != 0]

# Gerekli olmayan sütunları çıkarma (tarih ve metin sütunları)
columns_to_drop = ['Ekleme tarihi', 'İlk Tescil Tarihi', 'Son Muayene Tarihi']
data_cleaned = data.drop(columns=columns_to_drop)

# Aynı veriyi içeren sütunları tespit etme
nunique = data_cleaned.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index

# Bu sütunları çıkarma
data_cleaned = data_cleaned.drop(columns=cols_to_drop)

# "-" değerlerini NaN yapma
data_cleaned = data_cleaned.replace('-', np.nan)

# NaN değerlerini rastgele "B" veya "D" ile doldurma
data_cleaned = data_cleaned.apply(lambda x: x.fillna(np.random.choice(['B', 'D']) if x.dtype == 'object' else x))


In [ ]:
data.isnull().sum() # Missing data kontrolü

Açılış fiyatı             0
Kapanış fiyatı            0
Kaput                     0
SoLÖnÇamurluk             0
SoLÖnKapı                 0
                         ..
Hafızalı Koltuk           0
CD Çalarlı Radyo          0
Döşeme: Alcantara/Deri    0
4 X 4                     0
CD Değiştiricili Radyo    0
Length: 73, dtype: int64

In [ ]:
# Özellik ve hedef değişkenleri ayırma
X = data_cleaned.drop(columns=['Kapanış fiyatı'])
y = data_cleaned['Kapanış fiyatı']

# Kategorik sütunları one-hot encoding ile dönüştürme
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['number']).columns.tolist()

# ColumnTransformer ile sayısal ve kategorik sütunları dönüştürme
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Pipeline ile dönüşüm işlemlerini gerçekleştirme
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])


# Verileri dönüştürme ve eğitim-test setlerine ayırma
X_processed = pipeline.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, train_size=0.7, random_state=42)

In [ ]:
# MLP modelini oluşturma
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Model eğitimi
mlp.fit(X_train, y_train)

# Tahmin yapma
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)

# Eğitim ve test hatalarını hesaplama
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print("Eğitim MSE:", mse_train)
print("Test MSE:", mse_test)